In [1]:
import time
from bs4 import BeautifulSoup
import requests
import datetime
import pandas as pd
import smtplib
from email.message import EmailMessage
import os
from collections import defaultdict

In [2]:
now = datetime.datetime.now() # - datetime.timedelta(10)
now

datetime.datetime(2019, 4, 27, 17, 45, 19, 786009)

In [3]:
def get_date(date, offset=None):
    if offset:
        offset_days = datetime.timedelta(days=offset)
        date = date + offset_days
    year = date.year
    day = date.day
    month = date.month
    date = '{:04d}{:02d}{:02d}'.format(year, month, day)
    return date

In [4]:
check_days = 7

In [5]:
listing_dates = [get_date(now, offset=i) for i in range(check_days)]
listing_dates

['20190427',
 '20190428',
 '20190429',
 '20190430',
 '20190501',
 '20190502',
 '20190503']

In [6]:
destinations = {
    'Forum': 'https://in.bookmyshow.com/buytickets/pvr-forum-mall-koramangala/cinema-bang-PVBN-MT/',
}

In [7]:
watching = ['Avengers: Endgame']
watching

['Avengers: Endgame']

In [8]:
for dest, dest_url in destinations.items():
    dest_url = '{}/'.format(dest_url.rstrip('/'))
    destinations[dest] = dest_url
destinations

{'Forum': 'https://in.bookmyshow.com/buytickets/pvr-forum-mall-koramangala/cinema-bang-PVBN-MT/'}

In [9]:
history = pd.read_csv('history.csv')
history

,date,movie,dest
0,20190428,Avengers: Endgame,Forum


In [10]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

In [11]:
#gmail_user = os.environ['gmail_user']
gmail_user = 'bmsnotifier@gmail.com'
#gmail_password = os.environ['gmail_password']
gmail_password = 'bmsnotifier123'

try:  
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.login(gmail_user, gmail_password)
except:  
    raise ValueError('Something went wrong while connecting to gmail...')
    
emails = ['singhsartaj94@gmail.com']#, 'dhruvgoel2295@gmail.com', '4bhimanyu@gmail.com', 'siddharthsikri46@gmail.com']

In [14]:
def notify(emails, movies): 
    movie_names = ','.join(movies.keys())
    subject = 'New Listings for {}'.format(movie_names)
    
    body = ''
    for movie_name in movies.keys():
        body += '{}\n'.format(movie_name)
        for date, dest, times in movies[movie_name]:
            times = ' '.join(times)
            day = pd.to_datetime(date).day_name()
            body += '{}, {}, {} => {}\n\n'.format(dest, day, date, times)
    msg = EmailMessage()
    msg.set_content(body)
    msg['Subject'] = subject
    msg['From'] = gmail_user
    msg['To'] = emails

    server.send_message(msg)

In [15]:
def bms_scrapper(watching, destinations, listing_dates, history):
    movies = defaultdict(list)
    for date in listing_dates:
        frm_date = '{}-{}-{}'.format(date[-2:], date[4:6], date[:4])
        for movie_name in watching:
            for dest, dest_url in destinations.items():
                logged = history.loc[(history['date'] == int(date))&(history['movie'] == movie_name)&(history['dest'] == dest), :]
                if logged.shape[0] != 0:
                    print('Already Notified for {}, {}'.format(frm_date, movie_name))
                    continue
                movie_date_url = '{}{}'.format(dest_url, date)
                time.sleep(1)
                movie_date_info = requests.get(movie_date_url, params=headers)
                if date not in movie_date_info.url:
                    print('{} not listed for {}'.format(movie_name, frm_date))
                    break
                movie_date_html = BeautifulSoup(movie_date_info.text)
                containers = movie_date_html.find_all('div', attrs={'class': 'container'})
                container = containers[2]
                available_times = []
                for movie in container.find_all('li', attrs={'class': 'list'}):
                    curr_movie_name = movie.find('strong').text
                    if movie_name not in curr_movie_name:
                        continue
                    body = movie.find('div', attrs={'class': 'body'})
                    listings = body.find_all('div')
                    available = filter(lambda listing: listing['data-oline'] == 'Y', listings)
                    available_times.extend(list(map(lambda listing: listing.text, available)))
                history = history.append([{'movie': movie_name, 'date': int(date), 'dest': dest}])
                if len(available_times):
                    movies[movie_name].append((frm_date, dest, available_times))
    notify(emails, movies)

In [16]:
bms_scrapper(watching, destinations, listing_dates, history)

Already Notified for 28-04-2019, Avengers: Endgame
Avengers: Endgame not listed for 01-05-2019
Avengers: Endgame not listed for 02-05-2019
Avengers: Endgame not listed for 03-05-2019


In [17]:
#history.to_csv('history.csv', index=False)